In [1]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.layers import Dense, GlobalAveragePooling2D, merge, Dropout
from keras import backend as K
import numpy as np
from keras.callbacks import Callback
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
import preprocess2 as pp
# a képek betöltése és előfeldolgozása
csv_data, author_stat = pp.csv_load()
# a 10 legtöbb képpel rendelkező alkotót kiválogatjuk
authors_to_select = author_stat[-10:]
authors_to_select

[('6f80666437feea42f295cdc0f1eb4df9', 389),
 ('0eeac4ecff259dc515be795e1a76019a', 390),
 ('a6027a4ba71b61a55ea598379c9d508c', 391),
 ('481c5c92d55717167e01821144a54635', 393),
 ('10bc951c2eb4a2f05fa773bdaace4e3b', 397),
 ('121fffad1eb6f7dff228b8a71b6aec72', 397),
 ('3f8dc381ccfe9d5cc88b75970262715b', 400),
 ('d8a3c897c506be7de91d8f892f14f934', 401),
 ('96e7b1bc8d52e18caf0af34fec2e9bcb', 402),
 ('3cc9a44380296d93e68b71a27643c25f', 413)]

In [3]:
csv_data = pp.csv_select(csv_data, authors_to_select)
# az útvonalnak az összes képet tartalmazó könyvtárra kell mutatnia, mert különben
# nem fog tudni belőle elég képet kiolvasni
train_images, labels = pp.load_images(csv_data, "/media/bence/121A62041A61E4E7/learn/train")

2000 images loaded
3973 images in the result


In [ ]:
# adatok megfelelő formátumra hozása a keras számára
encoder = preprocessing.LabelEncoder()
encoder.fit(labels)

encoded_l = encoder.transform(labels)
print(encoded_l)

labels_onehot = to_categorical(encoded_l)
print(labels_onehot)

In [ ]:
class TrainingHistory(Callback):
    # Tanulási folyamat elején létrehozunk egy-egy üres listát a kinyerni kívánt metrikák tárolása céljából.
    def on_train_begin(self, logs={}):
        # Hiba mértéke a tanító adatokon.
        self.losses = []
        # Hiba mértéke a validációs adatokon.
        self.valid_losses = []
        # A modell jóságát, pontosságát mérő mutatószám a tanító adatokon. 
        self.accs = []
        # A modell jóságát, pontosságát mérő mutatószám a validációs adatokon. 
        self.valid_accs = []
        # A tanítási fázisok sorszámozása.
        self.epoch = 0
    
    # Minden egyes tanítási fázis végén mentsük el, hogy hogyan teljesít aktuálisan a háló. 
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 1 == 0:
            self.losses.append(logs.get('loss'))
            self.valid_losses.append(logs.get('val_loss'))
            self.accs.append(logs.get('acc'))
            self.valid_accs.append(logs.get('val_acc'))
            self.epoch += 1
            
history = TrainingHistory()

In [ ]:
##########################################################
# error esetére, elvileg nem okoz gondot 'jó' esetben sem
import tensorflow as tf
tf.python.control_flow_ops = tf
##########################################################

# előtanított modell betöltése, a fully-connected rétegek nélkül
base_model = InceptionV3(weights='imagenet', include_top=False)
# az utolsó konvolúciós réteg utána egy global average pooling réteget teszünk, ez rögtön "lapítja" (flatten) a 2D konvolúciót


In [ ]:
# kinyerjük a stílusjegyeket a cnn köztes rétegegeiből (és max pool cnn kimeneti rétegére)
style1 = base_model.layers[54].output
style1 = GlobalAveragePooling2D()(style1)
style1 = Dense(96, activation='relu')(style1)
style2 = base_model.layers[117].output
style2 = GlobalAveragePooling2D()(style2)
style2 = Dense(160, activation='relu')(style2)
style3 = base_model.layers[184].output
style3 = GlobalAveragePooling2D()(style3)
style3 = Dense(320, activation='relu')(style3)

style_final = base_model.output
style_final = GlobalAveragePooling2D()(style_final)

# egymás mellé tesszük a különböző szintű feature-öket
ff = merge([style1, style2, style3, style_final], mode='concat')

# ezután hozzáadunk két előrecsatolt réteget ReLU aktivációs függvénnyel
ff = Dense(1024, activation='relu')(ff)
ff = Dropout(0.5)(ff)
ff = Dense(1024, activation='relu')(ff)

# és végül egy kimenete lesz a hálónak - a "binary_crossentropy" költségfüggvénynek erre van szüksége
predictions = Dense(labels_onehot.shape[1], activation='softmax')(ff)
# a model létrehozása
model = Model(input=base_model.input, output=predictions)

In [ ]:
# két lépésben fogjuk tanítani a hálót
# az első lépésben csak az előrecsatolt rétegeket tanítjuk, a konvolúciós rétegeket befagyasztjuk
for layer in base_model.layers:
    layer.trainable = False
# lefordítjuk a modelt (fontos, hogy ezt a rétegek befagyasztása után csináljuk"
# mivel két osztályunk van, ezért bináris keresztentrópia költségfüggvényt használunk
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_images, labels_onehot, batch_size=8, nb_epoch=50, validation_split=0.2, callbacks=[history])

In [ ]:
# ehhez először nézzük meg a háló felépítését
print("Az Inception V3 konvolúciós rétegei:")
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

In [ ]:
# majd a hálónak csak az első 172 rétegét fagyasztjuk, a többit pedig engedjük tanulni
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

In [ ]:
# ez után újra le kell fordítanunk a hálót, hogy most már az Inception V3 felsőbb rétegei tanuljanak
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# és ismét indítunk egy tanítást, ezúttal nem csak az előrecsatolt rétegek,
# hanem az Inception V3 felső rétegei is tovább tanulnak
model.fit(train_images, labels_onehot,  batch_size=16, nb_epoch=100, validation_split=0.2, callbacks=[history])
print("Tanítás vége.")

In [ ]:
predicted_values = model.predict(np.asarray(train_images)).shape

In [ ]:
max_pred = np.zeros(predicted_values.shape, dtype=int)
festok = []
for i, pred in enumerate(predicted_values):
    d = np.argmax(pred)
    max_pred[i, d] = 1
    festok.append(d)
festok

In [ ]:
encoder.inverse_transform(festok)

In [ ]:
# vissza kéne alakítani valahogy...
ecovered_X = np.array([ohc.active_features_[col] for col in out.sorted_indices().indices])\
.reshape(n_samples, n_features) - ohc.feature_indices_[:-1]